# Exercise 2: Deep learning

CSCS-ICS-DADSi Summer School: Accelerating Data Science with HPC
September 4 - 6, 2017
Swiss National Supercomputing Centre

Inspired from:

http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

We will use the PyTorch deep learning framework (http://pytorch.org/) in this exercise.

PyTorch is a Python-based reincarnation of the lua-based Torch framework (http://torch.ch/), with features such as dynamic graphs and general-purpose automatic differentiation.

Please visit the PyTorch website for more information and tutorials.

## Part 1: Tensors

We start by importing `torch`.

In [1]:
import torch

A note on GPU usage:
.cuda()

## Part 2: Automatic differentiation

In [ ]:
Variables, gradients

## Part 3: Neural networks

### Defining the network

### The loss function

### Optimization

### Training data